# Reciprocal space

This notebook is to accompany the _Reciprocal space_ content.

Version 0.1, updated 26/09/2021 by AJM

## Import packages

To streamline operations in Python, packages can be imported to perform a host of various tasks. To make this process as simple as possible, all the required packages are included in the file _[SSP.py](https://github.com/Andy-UTAS/Solid-state/blob/master/SSP.py)_ and thus we can import all of the content: 

In [ ]:
from SSP import *

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
# Define primitive lattice vectors
a1 = np.array([1,0])
a2 = np.array([0.5,np.sqrt(3)/2])
# Compute reciprocal lattice vectors
b1,b2 = np.linalg.inv(np.array([a1,a2]).T) @ np.eye(2)*2*np.pi

fig = make_subplots(rows=1, cols=2,shared_yaxes=True,subplot_titles=('Real Space', 'Reciprocal Space'))

# Generates the lattice given the lattice vectors
def lattice_generation(a1,a2,N):
    grid = np.arange(-N//2,N//2,1)
    xGrid, yGrid = np.meshgrid(grid,grid)
    return np.reshape(np.kron(xGrid.flatten(),a1),(-1,2))+np.reshape(np.kron(yGrid.flatten(),a2),(-1,2))


def subplot(a1,a2,col):
    N = 6
    lat_points = lattice_generation(a1,a2,N)
    line_a1 = np.transpose([[0,0],a1])
    line_a2 = np.transpose([[0,0],a2])
    dotLine_a1 = np.transpose([a1,a1+a2])
    dotLine_a2 = np.transpose([a2,a1+a2])


    fig.add_trace(
        go.Scatter(visible=False, x=line_a1[0],y=line_a1[1],mode='lines',line_color='red'
        ), row = 1, col = col
    )
    fig.add_trace(
        go.Scatter(visible=False, x=line_a2[0],y=line_a2[1], mode='lines',line_color='red'
        ), row = 1, col = col
    )
    fig.add_trace(
        go.Scatter(visible=False, x=dotLine_a1[0],y=dotLine_a1[1],mode='lines',line_color='red',line_dash='dot'
        ), row = 1, col = col
    )
    fig.add_trace(
        go.Scatter(visible=False, x=dotLine_a2[0],y=dotLine_a2[1],mode='lines',line_color='red',line_dash='dot'
        ), row = 1, col = col
    )

    fig.add_trace(
        go.Scatter(visible=False, x=lat_points.T[0],y=lat_points.T[1],mode='markers',marker=dict(
            color='Black',
            size = 10
            )
        ), row = 1, col = col
    )

# Generate subplots to be used by the slider
N_values = 10
for i in np.linspace(2.5,3.5,N_values):
    subplot(a1*i,a2*i,1)
    subplot(b1/i,b2/i,2)

# Define the default subplot
active = 4
for i in range(10):   
    fig.data[active*10+i].visible = True

steps = []
for i in range(N_values):
    step = dict(
        label = 'Lattice Constant',
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    for j in range(10):
        step["args"][1][i*10+j] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    tickcolor = 'White',
    font_color = 'White',
    currentvalue_font_color = 'Black',
    active = active,
    name = 'Lattice Constant',
    steps = steps
)]

# Annotating lattice vectors
def text_dict(text,pos,ref=('x1','y1')):
    dic = {
            'x' : pos[0],
            'y' : pos[1],
            'xref' : ref[0],
            'ayref' : ref[1],
            'text' : text,
            'font' : {
                'size' : 15,
                'color' : 'black'
            },
            'showarrow' : False
    }
    return dic
annotations = [a.to_plotly_json() for a in fig["layout"]["annotations"]]
# annotations.append(text_dict('$\mathbf{a}_1$',(1.5,-0.5)))
# annotations.append(text_dict('$\mathbf{a}_2$',(0,1.3)))
# annotations.append(text_dict('$\mathbf{b}_1$',(0.8,-1),('x2','y2')))
# annotations.append(text_dict('$\mathbf{b}_2$',(-0.6,1.2),('x2','y2')))
fig["layout"]["annotations"] = annotations


plot_range = 5
fig.update_layout(
    sliders=sliders,
    showlegend = False,
    plot_bgcolor = 'rgb(254, 254, 254)',
    width = 800,
    height = 400,
    xaxis = dict(
        range=[-plot_range,plot_range],
        visible = False,
        showgrid = False,
    ),
    yaxis = dict(
      range = [-plot_range,plot_range],
      visible = False,
      showgrid = False,
    )
)
fig.update_xaxes(range=[-plot_range, plot_range], row=1, col=2, visible=False)
fig.update_yaxes(row=1, col=2, scaleanchor="x", scaleratio=1)
fig.update_yaxes(row=1, col=1, scaleanchor="x", scaleratio=1)


if True:
    # html file
    py.plot(fig, filename='4-2-reciprocal.html')

fig.show()